In [ ]:
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# [TR-016] Complex integral

<!-- cspell:ignore ndim orthopy -->

```{autolink-skip}
```

In [ ]:
%pip install -q ndim==0.1.7 orthopy==0.9.6 quadpy==0.16.11 scipy==1.8.0

As noted in {doc}`/report/003`, {func}`scipy.integrate.quad` cannot handle complex integrals. In addition, one can get into trouble with vectorized input ({obj}`numpy.array`s) on a lambdified {class}`sympy.Integral <sympy.integrals.integrals.Integral>`. This report discusses both problems and proposes some solutions.

## Complex integration

SciPy cannot integrate complex functions:

In [ ]:
from scipy.integrate import quad


def integrand(x):
    return x * (x + 1j)


quad(integrand, 0.0, 2.0)

### Split real and imaginary integral

A [proposed solution](https://stackoverflow.com/a/5966088) is to wrap the {func}`~scipy.integrate.quad` function in a special integrate function that integrates the real and imaginary part of a function separately:

In [ ]:
import numpy as np


def complex_integrate(func, a, b, **quad_kwargs):
    def real_func(x):
        return np.real(func(x))

    def imag_func(x):
        return np.imag(func(x))

    real_integral, real_integral_err = quad(real_func, a, b, **quad_kwargs)
    imag_integral, imag_integral_err = quad(imag_func, a, b, **quad_kwargs)
    return (
        real_integral + 1j * imag_integral,
        real_integral_err**2 + 1j * imag_integral_err,
    )

In [ ]:
complex_integrate(integrand, 0.0, 2.0)

:::{warning}

The handling of uncertainties is incorrect.

:::

### Integrate with `quadpy`

[Alternatively](https://stackoverflow.com/a/42866568), one could use [`quadpy`](https://github.com/sigma-py/quadpy), which essentially does the same as in {ref}`report/016:Split real and imaginary integral`, but can also (to a large degree) handle vectorized input and properly handles uncertainties.

In [ ]:
import quadpy

quadpy.quad(integrand, a=0.0, b=2.0)

:::{note}

One may need to play around with the tolerance if the function is not smooth, see [sigma-py/quadpy#255](https://github.com/sigma-py/quadpy/issues/255).

:::

:::{tip}

[`quadpy`](https://github.com/sigma-py/quadpy) raises exceptions with {obj}`ModuleNotFoundError`s that are a bit unreadable. They are caused by the fact that [`orthopy`](https://pypi.org/project/orthopy) and [`ndim`](https://pypi.org/project/ndim) need to be installed separately.

:::

## Vectorized input

The dispersion integral from Eq.&nbsp;{eq}`integrand` in {doc}`/report/003` features a variable&nbsp;$s$ that is an argument to the function $\Sigma_a$. This becomes a problem when $s$ gets vectorized (in this case: gets an event-wise {obj}`numpy.array` of invariant masses). Here's a simplified version of the problem:

In [ ]:
from functools import partial


def parametrized_func(s_prime, s):
    return s_prime * (s_prime + s + 1j)


s_array = np.linspace(-1, 1, num=10)
quadpy.quad(
    partial(parametrized_func, s=s_array),
    a=0.0,
    b=2.0,
)

The way out seems to be to **vectorize the [`quadpy.quad()`](https://github.com/sigma-py/quadpy) call itself** and forward the function arguments through {func}`functools.partial`:

In [ ]:
from functools import partial


@np.vectorize
def vectorized_quad(func, a, b, **func_kwargs):
    return quadpy.quad(partial(func, **func_kwargs), a, b)


vectorized_quad(parametrized_func, a=0.0, b=2.0, s=s_array)

Note, however, that this becomes difficult to implement as {func}`~sympy.utilities.lambdify.lambdify` output for a {class}`sympy.Integral <sympy.integrals.integrals.Integral>`. An attempt at this is made in {doc}`/report/003`.